<a href="https://colab.research.google.com/github/sakshamnanda/ISY503-Assessment-3/blob/main/ISY503_A3_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# Import libs

!pip install anvil-uplink
import anvil.server
#connect the notebook withUplink script.
anvil.server.connect("FOYNAAKTKIEMX5ZYXBECJTFE-5MSVGBRZOX2ZH4PP")

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
import numpy as np
import pandas as pd
import string

from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords
from nltk.corpus import words

!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [46]:
# Read XML review files

positive_reviews = BeautifulSoup(open('/content/positive1.review').read(), features="html5lib")
positive_reviews = positive_reviews.findAll('review_text')
positive_reviews = [i.contents[0] for i in positive_reviews]

negative_reviews = BeautifulSoup(open('/content/negative1.review').read(), features="html5lib")
negative_reviews = negative_reviews.findAll('review_text')
negative_reviews = [i.contents[0] for i in negative_reviews]

In [48]:
# Frame it

data = {'Text': positive_reviews + negative_reviews, 'Sentiment': [1] * len(positive_reviews) + [0] * len(negative_reviews)}
df = pd.DataFrame(data = data)

In [49]:
# Define tests variables

X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Sentiment'], test_size=0.2, random_state=42)


In [50]:
wordnet_lemmatizer = WordNetLemmatizer()
english_stopwords = stopwords.words('english')

def isMisspelled(word):
  return word not in words.words()

def clear(sen, fix_misspelling = False):
    sen = sen.lower() # set all to downcase
    tokens = nltk.tokenize.word_tokenize(sen) # split string into tokens
    tokens = [t for t in tokens if len(t) >= 3] # remove words shorter than 3 chracters
    tokens = [t.translate(str.maketrans('', '', string.punctuation)) for t in tokens] # remove punctuation
    tokens = [t for t in tokens if not any(i.isdigit() for i in t)] # remove digits
    if fix_misspelling: # default false, for performance purposes
      tokens = [spell(t) for t in tokens if isMisspelled(t)] # check/fix misspelling
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in english_stopwords] # remove stopwords    
    return tokens

# encode into numerical
count_vect_model = CountVectorizer(tokenizer = clear)
count_vect_model.fit(X_train)

X_train_cnt = count_vect_model.transform(X_train)
X_test_cnt = count_vect_model.transform(X_test)

model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train_cnt, y_train)

print("Test Accuracy:", round(model.score(X_test_cnt, y_test), 3))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


Test Accuracy: 0.816


In [51]:
# Auxiliar class to compare future entries
class ValuedWord:
  def __init__(self, word, value):
    self.word = word
    self.value = value

valuedWords = []

# Examinating of the words that have the biggest impact on prediction
threshold = 0.8
for i, j in zip(count_vect_model.get_feature_names_out(), model.coef_[0]):
    if j > threshold or j < -threshold:
        vw = ValuedWord(i, round(j, 3))
        valuedWords.append(vw)

In [65]:
#Call the function to anvil 
@anvil.server.callable
def entryText(entrySen):
  # Splitting sentence into cleaned tokens
  entered_words = clear(entrySen, False)

  senValue = 0
  # Compare each word from the entered sentence with the those from the XML  
  for vw in valuedWords:
    for word in entered_words:
      if word == vw.word:
        print(vw.word, vw.value)
        senValue += vw.value

  # Sum up sentiments value
  if senValue != 0:
    if senValue > 0:
      return 'The sentence is positive!'
      
    else:
      return'The sentence is negative!'
     


In [ ]:
#Keeps the notebook running and  the app calls the functions indefinitely.
anvil.server.wait_forever()

love 1.178
love 1.178
bad -0.985
love 1.178
